<a href="https://colab.research.google.com/github/kimdahyeon977/CLAM/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/CycleGAN-PyTorch')

In [ ]:
import argparse
import itertools
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
from PIL import Image
import models  # models.py 코드를 임포트 해주고 있음
import utils
from eval import evaluate
import sys
import datetime
import time
import pandas as pd
import random
import numpy as np
torch.backends.cudnn.benchmark = True

In [ ]:
def initWeights(m):
    """
    어떤 모델 m의 가중치를 초기화 해주는 함수
    """
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:  # 합성곱 계층이 존재하면
        torch.nn.init.normal(m.weight.data, 0.0, 0.02)  # m의 합성곱 계층들의 가중치를 평균 0.0 표준편차 0.02인 정규 분포로 초기화
    elif classname.find('BatchNorm2d') != -1:  # BatchNorm2d가 존재하면
        torch.nn.init.normal(m.weight.data, 1.0, 0.02)  # 초기화를 평균 1.0 표준편차 0.02인 정규 분포로 초기화
        torch.nn.init.constant(m.bias.data, 0.0)  # 편향 값들은 0.0의 상수로 초기화

class LR_sched():
    def __init__(self, numEpochs, decayEpoch):
        assert ((numEpochs - decayEpoch) > 0), "ohh no, decay > number epochs"
        self.numEpochs = numEpochs
        self.decayEpoch = decayEpoch
    def step(self, currentEpoch):
        return 1.0 - max(0, currentEpoch - self.decayEpoch)/(self.numEpochs - self.decayEpoch)
#---------

#----------
class ImageBuffer():
    """
    이미지를 버퍼라고 하는 변수에 넣어뒀다가, 새로운 데이터가 들어오면 기존에 있던거는 출력하고, 새로운 데이터를 저장하는 코드
    """
    def __init__(self, size=50):
        self.size = size
        self.bufferSize = 0
        self.buffer = []
    def pushPop(self, data):
        if self.size == 0:
            return data
        returnData = []
        for element in data:
            element = torch.unsqueeze(element.data, 0)  # 데이터에 차원을 하나 늘려주는 코드
            if self.bufferSize < self.size:
                self.bufferSize +=  1
                self.buffer.append(element)
                returnData.append(element)
            else:
                p = random.uniform(0, 1)
                if p > 0.5:
                    random_id = random.randint(0, self.size - 1)
                    tmp = self.buffer[random_id].clone()
                    returnData.append(tmp)
                    self.buffer[random_id] = element
                else:
                    returnData.append(element)
        return torch.cat(returnData, 0)
#----------
class LossLogger():
    def __init__(self, numEpochs, numBatches):
        self.numEpochs =numEpochs
        self.numBatches = numBatches
        self.losses = {}
        self.timeStart = time.time()
        self.timeBatchAvg = 0

    def log(self, currentEpoch, currentBatch, losses):
        sys.stdout.write('\rEpoch %03d/%03d [%04d/%04d] | ' % (currentEpoch, self.numEpochs, currentBatch, self.numBatches))
        for lossName in losses:
            if lossName not in self.losses:
                self.losses[lossName] = []
                self.losses[lossName].append(losses[lossName].item())
            else:
                if len(self.losses[lossName]) < currentEpoch:
                    self.losses[lossName].append(losses[lossName].item())
                else:
                    self.losses[lossName][-1] += losses[lossName].item()
            sys.stdout.write('%s: %.4f | ' % (lossName, self.losses[lossName][-1]/currentBatch))
            if currentBatch % self.numBatches == 0 :
                self.losses[lossName][-1] *= 1./currentBatch

        batchesDone =  (currentEpoch-1)*self.numBatches + currentBatch
        self.timeBatchAvg = (time.time() - self.timeStart)/float(batchesDone)
        batchesLeft = self.numEpochs*self.numBatches - batchesDone
        sys.stdout.write('ETA: %s' % (datetime.timedelta(seconds=batchesLeft*self.timeBatchAvg)))

        if currentBatch % self.numBatches == 0 :
            sys.stdout.write('\n')

    def plot(self):
        for lossName in self.losses:
            plt.figure()
            plt.plot(range(len(self.losses[lossName])),self.losses[lossName])
            plt.title(lossName)
            plt.xlabel("Epoch")
            plt.ylabel("Loss")
            plt.savefig('output/'+lossName+'.png')

    def save(self):
        df = pd.DataFrame.from_dict(self.losses)
        df.to_csv("output/losses.csv")

In [ ]:
def initWeights(m):
    """
    어떤 모델 m의 가중치를 초기화 해주는 함수
    """
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:  # 합성곱 계층이 존재하면
        torch.nn.init.normal(m.weight.data, 0.0, 0.02)  # m의 합성곱 계층들의 가중치를 평균 0.0 표준편차 0.02인 정규 분포로 초기화
    elif classname.find('BatchNorm2d') != -1:  # BatchNorm2d가 존재하면
        torch.nn.init.normal(m.weight.data, 1.0, 0.02)  # 초기화를 평균 1.0 표준편차 0.02인 정규 분포로 초기화
        torch.nn.init.constant(m.bias.data, 0.0)  # 편향 값들은 0.0의 상수로 초기화

class LR_sched():
    def __init__(self, numEpochs, decayEpoch):
        assert ((numEpochs - decayEpoch) > 0), "ohh no, decay > number epochs"
        self.numEpochs = numEpochs
        self.decayEpoch = decayEpoch
    def step(self, currentEpoch):
        return 1.0 - max(0, currentEpoch - self.decayEpoch)/(self.numEpochs - self.decayEpoch)
#---------

#----------
class ImageBuffer():
    """
    이미지를 버퍼라고 하는 변수에 넣어뒀다가, 새로운 데이터가 들어오면 기존에 있던거는 출력하고, 새로운 데이터를 저장하는 코드
    """
    def __init__(self, size=50):
        self.size = size
        self.bufferSize = 0
        self.buffer = []
    def pushPop(self, data):
        if self.size == 0:
            return data
        returnData = []
        for element in data:
            element = torch.unsqueeze(element.data, 0)  # 데이터에 차원을 하나 늘려주는 코드
            if self.bufferSize < self.size:
                self.bufferSize +=  1
                self.buffer.append(element)
                returnData.append(element)
            else:
                p = random.uniform(0, 1)
                if p > 0.5:
                    random_id = random.randint(0, self.size - 1)
                    tmp = self.buffer[random_id].clone()
                    returnData.append(tmp)
                    self.buffer[random_id] = element
                else:
                    returnData.append(element)
        return torch.cat(returnData, 0)
#----------
class LossLogger():
    def __init__(self, numEpochs, numBatches):
        self.numEpochs =numEpochs
        self.numBatches = numBatches
        self.losses = {}
        self.timeStart = time.time()
        self.timeBatchAvg = 0

    def log(self, currentEpoch, currentBatch, losses):
        sys.stdout.write('\rEpoch %03d/%03d [%04d/%04d] | ' % (currentEpoch, self.numEpochs, currentBatch, self.numBatches))
        for lossName in losses:
            if lossName not in self.losses:
                self.losses[lossName] = []
                self.losses[lossName].append(losses[lossName].item())
            else:
                if len(self.losses[lossName]) < currentEpoch:
                    self.losses[lossName].append(losses[lossName].item())
                else:
                    self.losses[lossName][-1] += losses[lossName].item()
            sys.stdout.write('%s: %.4f | ' % (lossName, self.losses[lossName][-1]/currentBatch))
            if currentBatch % self.numBatches == 0 :
                self.losses[lossName][-1] *= 1./currentBatch

        batchesDone =  (currentEpoch-1)*self.numBatches + currentBatch
        self.timeBatchAvg = (time.time() - self.timeStart)/float(batchesDone)
        batchesLeft = self.numEpochs*self.numBatches - batchesDone
        sys.stdout.write('ETA: %s' % (datetime.timedelta(seconds=batchesLeft*self.timeBatchAvg)))

        if currentBatch % self.numBatches == 0 :
            sys.stdout.write('\n')

    def plot(self):
        for lossName in self.losses:
            plt.figure()
            plt.plot(range(len(self.losses[lossName])),self.losses[lossName])
            plt.title(lossName)
            plt.xlabel("Epoch")
            plt.ylabel("Loss")
            plt.savefig('output/'+lossName+'.png')

    def save(self):
        df = pd.DataFrame.from_dict(self.losses)
        df.to_csv("output/losses.csv")

In [ ]:
parser = argparse.ArgumentParser()  # 하이퍼파라미터 설정
# 사용자로부터 하이퍼파라미터를 입력받는 역할
# add_argument 하나하나가 사용자에게 입력받을 하이퍼파라미터를 정의하는 부분
# parser.add_argument('--epoch', type=int, default=0, help='starting epoch')
parser.add_argument('--numEpochs', type=int, default=200, help='number of training epochs')
parser.add_argument('--batchSize', type=int, default=1, help='batch size')
parser.add_argument('--dataroot', type=str, default='../datasets_npy/train/', help='directory of the dataset')
parser.add_argument('--lr', type=float, default=0.0002, help='initial learning rate')
parser.add_argument('--decayEpoch', type=int, default=100, help='epoch to start linearly decaying the learning rate to 0')
parser.add_argument('--lambdaCyc_x', type=float, default=10.0, help='lambda for cycle loss (x -> y -> x)')
parser.add_argument('--lambdaCyc_y', type=float, default=10.0, help='lambda for cycle loss (y -> x -> y)')
parser.add_argument('--lambdaIdentity', type=float, default=5.0, help='lambda for identity loss')
parser.add_argument('--size', type=int, default=512, help='size of squared img to use (resize and crop)')
parser.add_argument('--input_nc', type=int, default=1, help='number of channels of input data')
parser.add_argument('--output_nc', type=int, default=1, help='number of channels of output data')
parser.add_argument('--cuda', action='store_true', default=True, help='use GPU computation')
parser.add_argument('--n_cpu', type=int, default=12, help='number of cpu threads to use during batch generation')
parser.add_argument('--saveEpochFrq', type=int, default=20, help='frequency of saving checkpoints at the end of epochs') # 몇에폭마다 output 저장할것인지
parser.add_argument('--manualSeed', action='store_true', help='use manual seed')
parser.add_argument('--seedNum', type=int, default=6, help='seed')
parser.add_argument('--imageBuffer', action='store_true', help='use an image buffer')
parser.add_argument('--evaluation_on', type=str, default="test", help="data to use in evaluation")

parser.add_argument('--numEpochs_total', type=int, default=None, help='number of total training epochs')
parser.add_argument('--continue_epoch', type=int, default=0, help="continue training from epoch #")
parser.add_argument('--genG', type=str, default='output/netG.pth', help='generator checkpoint file x->y')
parser.add_argument('--genF', type=str, default='output/netF.pth', help='generator checkpoint file y->x')
parser.add_argument('--netD_x', type=str, default='output/netD_x.pth', help='Discriminator checkpoint file D_x')
parser.add_argument('--netD_y', type=str, default='output/netD_y.pth', help='Discriminator checkpoint file D_y')
parser.add_argument('--discratio',type=int, default=5)
parser.add_argument('--sample_ratio',type=float, default=1)

_StoreAction(option_strings=['--sample_ratio'], dest='sample_ratio', nargs=None, const=None, default=1, type=<class 'float'>, choices=None, required=False, help=None, metavar=None)

In [ ]:
# opt = parser.parse_args(['--numEpochs', '40, '--batchSize', '4', '--decayEpoch', '100', '--lr', '0.0002', '--evaluation_on', 'train', '--numEpochs_total', '200',
#                          '--lambdaCyc_x', '25', '--lambdaCyc_y', '25', '--lambdaIdentity', '0.5', '--continue_epoch','21','--discratio','7'])  # 사용자에게 하이퍼파라미터 입력받아서 opt에 저장을 한 것 # EX : continue_epoch 20 num epoch 40
# print(opt)
# #-1028-2048 바꿔 보는것을 추천 색이이상 3d 슬라이스에  cbct나 spct 범위 같은지 확인

opt = parser.parse_args(['--numEpochs', '120', '--batchSize', '4', '--decayEpoch', '100', '--lr', '0.0002', '--evaluation_on', 'train', '--numEpochs_total', '200',
                         '--lambdaCyc_x', '25', '--lambdaCyc_y', '25', '--lambdaIdentity', '0.5', '--continue_epoch','85','--discratio','7', '--sample_ratio','0.5'])  # 사용자에게 하이퍼파라미터 입력받아서 opt에 저장을 한 것 # EX : continue_epoch 20 num epoch 40
print(opt)
#

## Preliminaries

In [ ]:
opt = parser.parse_args(['--numEpochs', '120', '--batchSize', '4', '--decayEpoch', '51', '--lr', '0.0002', '--evaluation_on', 'train', '--numEpochs_total', '200', #evalution_on test 로 하면 test 평
                         '--lambdaCyc_x', '15', '--lambdaCyc_y', '15', '--lambdaIdentity', '2', '--continue_epoch','112','--discratio','9', '--sample_ratio','1',"--dataroot", "./datasets_npy",'--size','512'])  # 사용자에게 하이퍼파라미터 입력받아서 opt에 저장을 한 것 # EX : continue_epoch 20 num epoch 40
print(opt)
#decayEpoch - 특정 에폭까지는 러닝레이트 쓰다가 decay부터는 학습 낮춰준다

Namespace(numEpochs=120, batchSize=4, dataroot='./datasets_npy', lr=0.0002, decayEpoch=51, lambdaCyc_x=15.0, lambdaCyc_y=15.0, lambdaIdentity=2.0, size=512, input_nc=1, output_nc=1, cuda=True, n_cpu=12, saveEpochFrq=20, manualSeed=False, seedNum=6, imageBuffer=False, evaluation_on='train', numEpochs_total=200, continue_epoch=112, genG='output/netG.pth', genF='output/netF.pth', netD_x='output/netD_x.pth', netD_y='output/netD_y.pth', discratio=9, sample_ratio=1.0)


## Code

In [ ]:
if opt.numEpochs_total is None:
    opt.numEpochs_total = opt.numEpochs

if opt.manualSeed:
    torch.cuda.manual_seed(opt.seedNum)
    torch.cuda.manual_seed_all(opt.seedNum)

# 모델을 선언해주는 코드
G = models.Generator(opt.input_nc, opt.output_nc) #generator x->y
F = models.Generator(opt.output_nc, opt.input_nc) #generator y->x
D_x = models.Discriminator(opt.input_nc) #discriminator X
D_y = models.Discriminator(opt.input_nc) #discriminator Y

if opt.cuda:  # CUDA를 사용할 것이면?
    G.cuda()  # 각 모델을 GPU로 보내주는 코드
    F.cuda()
    D_x.cuda()
    D_y.cuda()

if opt.continue_epoch == 0:  # 처음부터 학습할 경우
    G.apply(initWeights)
    F.apply(initWeights)
    D_x.apply(initWeights)
    D_y.apply(initWeights)
else:  # 학습을 이어서 진행할 경우
    print("학습 재개")
    G.load_state_dict(torch.load(opt.genG))
    F.load_state_dict(torch.load(opt.genF))
    D_x.load_state_dict(torch.load(opt.netD_x))
    D_y.load_state_dict(torch.load(opt.netD_y))


# loss를 선언하는 부분
criterionGAN = torch.nn.MSELoss()  # 파이토치에 기본으로 탑재된 Loss들을 사용
criterionCycle = torch.nn.L1Loss()
criterionIdentity = torch.nn.L1Loss()

#   optim
optimizer_Generators = torch.optim.AdamW(itertools.chain(G.parameters(), F.parameters()),
                                lr=opt.lr, betas=(0.5, 0.999))
optimizer_D_x = torch.optim.AdamW(D_x.parameters(), lr=opt.lr*0.1, betas=(0.5, 0.999))    #discrminator lr generator 의 1/10 으로 학습
optimizer_D_y = torch.optim.AdamW(D_y.parameters(), lr=opt.lr*0.1, betas=(0.5, 0.999))

# 러닝레이트 스케쥴링
# lrScheduler_Generators = torch.optim.lr_scheduler.LambdaLR(optimizer_Generators, lr_lambda=LR_sched(opt.numEpochs_total, opt.decayEpoch).step)
# lrScheduler_D_x = torch.optim.lr_scheduler.LambdaLR(optimizer_D_x, lr_lambda=LR_sched(opt.numEpochs_total, opt.decayEpoch).step)
# lrScheduler_D_y = torch.optim.lr_scheduler.LambdaLR(optimizer_D_y, lr_lambda=LR_sched(opt.numEpochs_total, opt.decayEpoch).step)

### --- ADDED:
import torch.optim as optim

lrScheduler_Generators = optim.lr_scheduler.CosineAnnealingLR(optimizer_Generators, T_max=opt.numEpochs_total)
lrScheduler_D_x = optim.lr_scheduler.CosineAnnealingLR(optimizer_D_x, T_max=opt.numEpochs_total)
lrScheduler_D_y = optim.lr_scheduler.CosineAnnealingLR(optimizer_D_y, T_max=opt.numEpochs_total)
### --- end of ADDED

# 학습 진행한 만큼 lr Scheduler step 반영
for epoch in range(opt.continue_epoch):
    lrScheduler_Generators.step()
    lrScheduler_D_x.step()
    lrScheduler_D_y.step()

Tensor = torch.cuda.FloatTensor if opt.cuda else torch.Tensor

input_x = Tensor(opt.batchSize, opt.input_nc, opt.size, opt.size)
input_y = Tensor(opt.batchSize, opt.input_nc, opt.size, opt.size)
targetReal = Variable(Tensor(opt.batchSize).fill_(1.0), requires_grad=False)
targetFake = Variable(Tensor(opt.batchSize).fill_(0.0), requires_grad=False)

if opt.imageBuffer:
    bufferFake_x = ImageBuffer()
    bufferFake_y = ImageBuffer()

transformList = [
    transforms.ToTensor(),
    #transforms.RandomCrop(opt.size//2),
    transforms.Resize(opt.size),
    # transforms.RandomHorizontalFlip(),   #vertical and horizental 한개만
    #transforms.RandomVerticalFlip(),                                                     # 파이토치 텐서로 만들어주고 값 범위를 0~1로 조정          #크롭하면 리사이즈 주석해줘야한다
    ### --- ADDED:
    # transforms.RandomCrop(opt.size),
    # transforms.RandomHorizontalFlip(),
    ### --- end of ADDED
    transforms.Normalize((0.5), (0.5)),  # (x-0.5)/0.5 해서 범위를 -1~1로 조정
    #transforms.Resize(opt.size)
]




dataset = DataLoader(utils.LoadDataset('../datasets_npy', transformList=transformList, sample_ratio=opt.sample_ratio),
                        batch_size=opt.batchSize, shuffle=True, num_workers=opt.n_cpu, drop_last=True)





# 학습 경과를 기록하는 코드
logger = LossLogger(opt.numEpochs, len(dataset))

학습 재개


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
<ipython-input-13-17f683b3eb72>:65: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at /pytorch/torch/csrc/tensor/python_tensor.cpp:78.)
  input_x = Tensor(opt.batchSize, opt.input_nc, opt.size, opt.size)
환자별 이미지 적재 중: 100%|██████████| 17/17 [00:17<00:00,  1.04s/it]

환자 17명 | CBCT 1022장 | SIM 855장 확인
sample ratio: 1.0 | CBCT 855장 | SIM 855장 적재



/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
EXP_NAME="Quantification"
try:
  log_csv = pd.read_csv(f"{EXP_NAME}.csv")
except:
  log_csv = pd.DataFrame(columns=["epoch","Loss_Gen_GAN","Loss_Gen_Cycle","Loss_Gen_Identity","Loss_Gen","Loss_Disc","MAE","RMSE","PSNR","SSIM"])

In [ ]:
log_csv

,epoch,Loss_Gen_GAN,Loss_Gen_Cycle,Loss_Gen_Identity,Loss_Gen,Loss_Disc,MAE,RMSE,PSNR,SSIM


In [ ]:
# MAE evaluation = 동일한 slice , ROI 평가 ,   학습 = 중간지점에서 0.2 .0.3  몇장 ,

In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict
#   --------------------- TRAIN
for epoch in range(opt.continue_epoch, opt.numEpochs+1):
    losses = defaultdict(list) #딕셔너리를 만들건데 , 이 딕셔너리의 기본값은 비어있는 리스트로 설정한다
    for i, batch in enumerate(tqdm(dataset)):  # 데이터셋에서 batch_size 개의 데이터 불러오기
        currentBatch_x = Variable(input_x.copy_(batch['x']))  # 데이터를 불러와서, Variable로 감싸주고
        currentBatch_y = Variable(input_y.copy_(batch['y']))

        # import pdb;pdb.set_trace()

        # 데이터를 모델에 입력
        fake_y = G(currentBatch_x) #G(x)
        fake_x = F(currentBatch_y) #F(y)

        #-------- Generators loss
        optimizer_Generators.zero_grad()  # 옵티마이저를 초기화

        #lsgan loss
        lossGAN_G = criterionGAN(D_y(fake_y).squeeze(), targetReal.squeeze())  # (D_y(G(x)) - 1)^2
        lossGAN_F = criterionGAN(D_x(fake_x).squeeze(), targetReal.squeeze())  # (D_x(F(y)) - 1)^2

        #cycle loss
        recovered_x = F(fake_y) # F(G(x))
        recovered_y = G(fake_x) # G(F(y))
        lossCyc_x = criterionCycle(recovered_x, currentBatch_x) # | F(G(X)) - x |
        lossCyc_y = criterionCycle(recovered_y, currentBatch_y) # | G(F(y)) - y |
        lossCyc = lossCyc_x*opt.lambdaCyc_x + lossCyc_y*opt.lambdaCyc_y

        #identity loss
        lossId_x = criterionIdentity(F(currentBatch_x), currentBatch_x) # | F(x) - x |
        lossId_y = criterionIdentity(G(currentBatch_y), currentBatch_y) # | G(y) - y |
        lossId = (lossId_x + lossId_y)*opt.lambdaIdentity

        #total generators loss
        loss_Generators = lossGAN_G + lossGAN_F + lossCyc + lossId
        loss_Generators.backward()

        optimizer_Generators.step()

        #-------- Discriminator loss
        optimizer_D_x.zero_grad()  # optimizer 초기화 해주고
        # Loss 계산
        if opt.imageBuffer:
            lossGAN_D_x = (criterionGAN(D_x(currentBatch_x).squeeze(), targetReal.squeeze()) + criterionGAN(D_x(bufferFake_x.pushPop(fake_x).detach()).squeeze(), targetFake.squeeze()))*0.5 #(D_x(x)-1)^2 + (D_x(F(y)))^2
        else:
            lossGAN_D_x = (criterionGAN(D_x(currentBatch_x).squeeze(), targetReal.squeeze()) + criterionGAN(D_x(fake_x.detach()).squeeze(), targetFake.squeeze()))*0.5 #(D_x(x)-1)^2 + (D_x(F(y)))^2
        lossGAN_D_x.backward()  # 그래디언트 구하기
        if i%opt.discratio==0: #5번학습할때 1번 discr 학습
           optimizer_D_x.step()  # update!

        optimizer_D_y.zero_grad()  # optimizer 초기화
        if opt.imageBuffer:
            lossGAN_D_y = (criterionGAN(D_y(currentBatch_y).squeeze(), targetReal.squeeze()) + criterionGAN(D_y(bufferFake_y.pushPop(fake_y).detach()).squeeze(), targetFake.squeeze()))*0.5 #(D_y(y)-1)^2 + (D_y(G(x)))^2
        else:
            lossGAN_D_y = (criterionGAN(D_y(currentBatch_y).squeeze(), targetReal.squeeze()) + criterionGAN(D_y(fake_y.detach()).squeeze(), targetFake.squeeze()))*0.5 #(D_y(y)-1)^2 + (D_y(G(x)))^2
        lossGAN_D_y.backward()
        if i%opt.discratio==0: #5번학습할때 1번 discr 학습
           optimizer_D_y.step() # 똑같음

        # 학습 끝

        # 로스값들 저장
        losses["Loss_Gen"] += [loss_Generators.item()]
        losses["Loss_Gen_GAN"] += [lossGAN_G.item() + lossGAN_F.item()]
        losses["Loss_Gen_Identity"] += [lossId.item()]
        losses["Loss_Gen_Cycle"] += [lossCyc.item()]
        losses["Loss_Disc"] += [lossGAN_D_x.item() + lossGAN_D_y.item()]

    for k , v in losses.items():  #지금까지 기록된 step별 loss의

        losses[k]=np.mean(v)

        # losses = {'loss_Gen': loss_Generators.item(),
        #           'loss_Gen_GAN': (lossGAN_G + lossGAN_F).item(),
        #             'loss_Gen_identity': lossId.item(),
        #             'loss_Gen_cycle': (lossCyc).item(),
        #             'loss_Disc': (lossGAN_D_x + lossGAN_D_y).item()}
        # print(losses)

    # eval_results = evaluate(G, opt)
    # mae = eval_results["mae"]
    # rmse = eval_results["rmse"]
    # psnr = eval_results["psnr"]
    # ssim = eval_results["ssim"]
    print(f"""Epoch: {epoch+1}/{opt.numEpochs} Loss_Gen_GAN: {losses["Loss_Gen_GAN"]:.4f} Loss_Gen_Cycle: {losses["Loss_Gen_Cycle"]:.4f} Loss_Gen_IDT: {losses["Loss_Gen_Identity"]:.4f} Loss_Gen: {losses["Loss_Gen"]:.4f} Loss_Disc:{losses["Loss_Disc"]:.4f}""")
    # print(f"MAE: {mae:.4f}\t RMSE: {rmse:.4f}\t PSNR: {psnr:.4f}\t SSIM: {ssim:.4f}")

    plt.figure(figsize=(5,5))
    img = ((fake_y.detach().cpu()[0].permute(1, 2, 0).numpy() * 0.5) + 0.5) * 255
    img = img.astype(int)
    plt.axis('off')
    plt.imshow(img, cmap="gray")
    plt.show()

    # with open("./output/log.txt", "a") as f:
    #     print(f"Epoch: {epoch+1}/{opt.numEpochs} Loss_Gen_GAN: {(lossGAN_G + lossGAN_F):.4f} Loss_Gen_Cycle: {lossCyc:.4f} Loss_Gen_IDT: {lossId:.4f} Loss_Gen: {loss_Generators:.4f} Loss_Disc:{(lossGAN_D_x + lossGAN_D_y):.4f}")
    #     print(f"MAE: {mae}\t RMSE: {rmse}\t PSNR: {psnr}\t SSIM: {ssim}\t", file=f)

#저장코드
    log = {'Loss_Gen': losses["Loss_Gen"],
            'Loss_Gen_GAN': losses["Loss_Gen_GAN"],
            'Loss_Gen_Identity': losses["Loss_Gen_Identity"],
            'Loss_Gen_Cycle': losses["Loss_Gen_Cycle"],
            'Loss_Disc': losses["Loss_Disc"],
            'epoch': epoch,
            # 'MAE': mae,
            # 'RMSE': rmse,
            # 'PSNR': psnr,
            # 'SSIM': ssim
           }
    log=pd.DataFrame([log])
    log_csv =pd.concat([log_csv,log],ignore_index=True)
    log_csv.to_csv(f"{EXP_NAME}.csv", index=False) #로그 파일로 저장

    lrScheduler_Generators.step()
    lrScheduler_D_x.step()
    lrScheduler_D_y.step()

    # Save models
    if epoch % opt.saveEpochFrq == 0:  # saveEpochFrq 마다 중간 저장
        label = '_ep'+str(epoch)
        torch.save(G.state_dict(), 'output/netG'+label+'.pth')
        torch.save(F.state_dict(), 'output/netF'+label+'.pth')
        torch.save(D_x.state_dict(), 'output/netD_x'+label+'.pth')
        torch.save(D_y.state_dict(), 'output/netD_y'+label+'.pth')

    torch.save(G.state_dict(), 'output/netG.pth')
    torch.save(F.state_dict(), 'output/netF.pth')
    torch.save(D_x.state_dict(), 'output/netD_x.pth')
    torch.save(D_y.state_dict(), 'output/netD_y.pth')



  0%|          | 0/213 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  0%|          | 0/213 [00:15<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 64.12 MiB is free. Process 5184 has 14.68 GiB memory in use. Of the allocated memory 14.43 GiB is allocated by PyTorch, and 127.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
!python eval.py --batchSize 1 --dataroot ./datasets/cbct2sct/ --size 512 --input_nc 1 --output_nc 1 --cuda --genG output/netG.pth